<a href="https://colab.research.google.com/github/open-mmlab/mmaction2/blob/master/demo/mmaction2_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Dependencies

In [1]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2017 NVIDIA Corporation
Built on Fri_Nov__3_21:07:56_CDT_2017
Cuda compilation tools, release 9.1, V9.1.85
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [48]:

# Install some optional requirements
# !pip install -r requirements/optional.txt

     |████████████████████████████████| 37.2 MB 19.9 MB/s eta 0:00:01
     |████████████████████████████████| 14.1 MB 55.1 MB/s eta 0:00:01
     |████████████████████████████████| 948 kB 52.5 MB/s eta 0:00:01
     |████████████████████████████████| 183 kB 55.8 MB/s eta 0:00:01
     |████████████████████████████████| 297 kB 53.8 MB/s eta 0:00:01
     |████████████████████████████████| 388 kB 55.0 MB/s eta 0:00:01
     |████████████████████████████████| 14.5 MB 49.3 MB/s eta 0:00:01
     |████████████████████████████████| 4.1 MB 57.1 MB/s eta 0:00:01
     |████████████████████████████████| 12.4 MB 48.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 52.9 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 52.3 MB/s eta 0:00:01
     |████████████████████████████████| 148 kB 54.7 MB/s eta 0:00:01
     |████████████████████████████████| 4.4 MB 52.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 54.4 MB/s eta 0:00:01
     |████████████████████████

In [2]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMAction2 installation
import mmaction
print(mmaction.__version__)

# Check MMCV installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

import sys
sys.path.append('/home/actrec/.local/lib/python3.6/site-packages/decord-0.5.3-py3.6-linux-x86_64.egg')
import decord

1.7.0+cu110 True
0.12.0
11.0
GCC 7.5


In [4]:
cd ..

/home/actrec/.virtualenvs/mmaction/mmaction2


In [1]:
ls

 checkpoints/
 configs/
 demo.gif
 demo_gradcam.gif
 demo_gradcam.py
 demo.ipynb
 demo.mp4*
 demo_out.mp4
 demo.py
 demo_spatiotemporal_det.py
 faster_rcnn_r50_fpn_2x_coco.py
 fuse/
 label_map_ava.txt
 label_map_k400.txt
 long_video_demo.py
 mmaction2_tutorial.ipynb
 my-mmaction.ipynb
 README.md
'Split by Folders and Annotation Files creation.ipynb'
 webcam_demo.py


# TSN 90.48% 

In [11]:
from mmcv import Config
cfg = Config.fromfile('./configs/recognition/tsn/tsn_r50_video_1x1x8_100e_kinetics400_rgb.py')

In [25]:
# !mkdir checkpoints
# !wget -c https://download.openmmlab.com/   .pth \
#       -O checkpoints/db3c461.pth

--2021-03-22 01:46:07--  https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_video_1x1x8_100e_kinetics600_rgb/tsn_r50_video_1x1x8_100e_kinetics600_rgb_20201015-4db3c461.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.75.20.25
Connecting to download.openmmlab.com (download.openmmlab.com)|47.75.20.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99220779 (95M) [application/octet-stream]
Saving to: ‘checkpoints/tsn_r50_video_1x1x8_100e_kinetics600_rgb_20201015-4db3c461.pth’

checkpoints/tsn_r50 100%[===================>]  94,62M  10,5MB/s    in 11s     

2021-03-22 01:46:22 (8,61 MB/s) - ‘checkpoints/tsn_r50_video_1x1x8_100e_kinetics600_rgb_20201015-4db3c461.pth’ saved [99220779/99220779]



In [12]:
from mmcv.runner import set_random_seed

# Modify dataset type and path
cfg.dataset_type = 'VideoDataset'
cfg.data_root = 'data/childact_split/train/'
cfg.data_root_val = 'data/childact_split/val/'
cfg.ann_file_train = 'data/childact_split/childact_train_video.txt'
cfg.ann_file_val = 'data/childact_split/childact_val_video.txt'
cfg.ann_file_test = 'data/childact_split/childact_test_video.txt'

cfg.data.test.type = 'VideoDataset'
cfg.data.test.ann_file = 'data/childact_split/childact_test_video.txt'
cfg.data.test.data_prefix = 'data/childact_split/test/'

cfg.data.train.type = 'VideoDataset'
cfg.data.train.ann_file = 'data/childact_split/childact_train_video.txt'
cfg.data.train.data_prefix = 'data/childact_split/train/'

cfg.data.val.type = 'VideoDataset'
cfg.data.val.ann_file = 'data/childact_split/childact_val_video.txt'
cfg.data.val.data_prefix = 'data/childact_split/val/'

# The flag is used to determine whether it is omnisource training
cfg.setdefault('omnisource', False)
# Modify num classes of the model in cls_head
cfg.model.cls_head.num_classes = 7
# We can use the pre-trained TSN model
# cfg.load_from = './checkpoints/best_top1_acc_epoch_5.pth'
cfg.resume_from = './childact-mm/latest.pth'

# Set up working dir to save files and logs.
cfg.work_dir = './childact-mm/'

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.data.videos_per_gpu = 32
cfg.optimizer.lr = 0.0001
# cfg.lr_config.type = 'cyclic'
cfg.total_epochs = 30

# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 2
# We can set the log print interval to reduce the the times of printing log
cfg.log_config.interval = 10

# Set seed thus the results are more reproducible
cfg.seed = 42
set_random_seed(42, deterministic=False)
cfg.gpu_ids = range(1)

# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')


Config:
model = dict(
    type='Recognizer2D',
    backbone=dict(
        type='ResNet',
        pretrained='torchvision://resnet50',
        depth=50,
        norm_eval=False),
    cls_head=dict(
        type='TSNHead',
        num_classes=7,
        in_channels=2048,
        spatial_type='avg',
        consensus=dict(type='AvgConsensus', dim=1),
        dropout_ratio=0.4,
        init_std=0.01),
    train_cfg=None,
    test_cfg=dict(average_clips=None))
optimizer = dict(type='SGD', lr=0.0001, momentum=0.9, weight_decay=0.0001)
optimizer_config = dict(grad_clip=dict(max_norm=40, norm_type=2))
lr_config = dict(policy='step', step=[40, 80])
total_epochs = 30
checkpoint_config = dict(interval=2)
log_config = dict(interval=10, hooks=[dict(type='TextLoggerHook')])
dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = None
resume_from = './childact-mm/latest.pth'
workflow = [('train', 1)]
dataset_type = 'VideoDataset'
data_root = 'data/childact_split/train/'
data_root_val = 'dat

### Train a new recognizer

Finally, lets initialize the dataset and recognizer, then train a new recognizer!

In [13]:
import os.path as osp

from mmaction.datasets import build_dataset
from mmaction.models import build_model
from mmaction.apis import train_model

import mmcv

# Build the dataset
datasets = [build_dataset(cfg.data.train)]

# Build the recognizer
model = build_model(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# model = build_model(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# model = build_detector(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# model = build_detector(cfg.model, train_cfg=cfg.train_cfg, test_cfg=cfg.test_cfg)

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_model(model, datasets, cfg, distributed=False, validate=True)

2021-03-22 16:04:14,045 - mmaction - INFO - These parameters in pretrained checkpoint are not loaded: {'fc.bias', 'fc.weight'}


Use load_from_torchvision loader


2021-03-22 16:04:16,416 - mmaction - INFO - load checkpoint from ./childact-mm/latest.pth
2021-03-22 16:04:16,417 - mmaction - INFO - Use load_from_local loader
2021-03-22 16:04:16,539 - mmaction - INFO - resumed epoch 12, iter 561
2021-03-22 16:04:16,541 - mmaction - INFO - Start running, host: actrec@actrec-HP-Z4-G4-Workstation, work_dir: /home/actrec/.virtualenvs/mmaction/mmaction2/childact-mm
2021-03-22 16:04:16,542 - mmaction - INFO - workflow: [('train', 1)], max: 30 epochs
/home/actrec/.virtualenvs/mmaction/mmaction2/mmaction/core/evaluation/eval_hooks.py:131: UserWarning: runner.meta is None. Creating a empty one.
  warnings.warn('runner.meta is None. Creating a empty one.')
2021-03-22 16:05:56,931 - mmaction - INFO - Epoch [13][10/33]	lr: 5.000e-04, eta: 1:10:06, time: 10.039, data_time: 9.158, memory: 21448, top1_acc: 0.9375, top5_acc: 1.0000, loss_cls: 0.2199, loss: 0.2199, grad_norm: 3.0833
2021-03-22 16:07:05,519 - mmaction - INFO - Epoch [13][20/33]	lr: 5.000e-04, eta: 0:

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 126/126, 3.5 task/s, elapsed: 35s, ETA:     0s

2021-03-22 16:19:08,703 - mmaction - INFO - Evaluating top_k_accuracy ...
2021-03-22 16:19:08,705 - mmaction - INFO - 
top1_acc	0.8492
top5_acc	1.0000
2021-03-22 16:19:08,706 - mmaction - INFO - Evaluating mean_class_accuracy ...
2021-03-22 16:19:08,707 - mmaction - INFO - 
mean_acc	0.8492
2021-03-22 16:19:09,000 - mmaction - INFO - Now best checkpoint is saved as best_top1_acc_epoch_15.pth.
2021-03-22 16:19:09,001 - mmaction - INFO - Best top1_acc is 0.8492 at 15 epoch.
2021-03-22 16:19:09,002 - mmaction - INFO - Epoch(val) [15][33]	top1_acc: 0.8492, top5_acc: 1.0000, mean_class_accuracy: 0.8492
2021-03-22 16:20:54,290 - mmaction - INFO - Epoch [16][10/33]	lr: 5.000e-04, eta: 0:44:12, time: 10.529, data_time: 9.656, memory: 21448, top1_acc: 0.8656, top5_acc: 1.0000, loss_cls: 0.3135, loss: 0.3135, grad_norm: 5.0392
2021-03-22 16:22:01,583 - mmaction - INFO - Epoch [16][20/33]	lr: 5.000e-04, eta: 0:42:08, time: 6.729, data_time: 5.830, memory: 21448, top1_acc: 0.9094, top5_acc: 0.9969,

...

2021-03-22 16:43:42,967 - mmaction - INFO - Evaluating top_k_accuracy ...
2021-03-22 16:43:42,969 - mmaction - INFO - 
top1_acc	0.8571
top5_acc	1.0000
2021-03-22 16:43:42,970 - mmaction - INFO - Evaluating mean_class_accuracy ...
2021-03-22 16:43:42,971 - mmaction - INFO - 
mean_acc	0.8571
2021-03-22 16:43:43,294 - mmaction - INFO - Now best checkpoint is saved as best_top1_acc_epoch_20.pth.
2021-03-22 16:43:43,295 - mmaction - INFO - Best top1_acc is 0.8571 at 20 epoch.
2021-03-22 16:43:43,296 - mmaction - INFO - Epoch(val) [20][33]	top1_acc: 0.8571, top5_acc: 1.0000, mean_class_accuracy: 0.8571
2021-03-22 16:45:20,813 - mmaction - INFO - Epoch [21][10/33]	lr: 5.000e-04, eta: 0:21:02, time: 9.751, data_time: 8.872, memory: 21448, top1_acc: 0.8875, top5_acc: 1.0000, loss_cls: 0.2959, loss: 0.2959, grad_norm: 5.2473
2021-03-22 16:46:30,437 - mmaction - INFO - Epoch [21][20/33]	lr: 5.000e-04, eta: 0:19:34, time: 6.962, data_time: 6.085, memory: 21448, top1_acc: 0.8875, top5_acc: 1.0000, 

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 126/126, 3.5 task/s, elapsed: 36s, ETA:     0s

2021-03-22 17:08:12,494 - mmaction - INFO - Evaluating top_k_accuracy ...
2021-03-22 17:08:12,496 - mmaction - INFO - 
top1_acc	0.8571
top5_acc	1.0000
2021-03-22 17:08:12,496 - mmaction - INFO - Evaluating mean_class_accuracy ...
2021-03-22 17:08:12,497 - mmaction - INFO - 
mean_acc	0.8571
2021-03-22 17:08:12,497 - mmaction - INFO - Epoch(val) [25][33]	top1_acc: 0.8571, top5_acc: 1.0000, mean_class_accuracy: 0.8571
2021-03-22 17:09:53,555 - mmaction - INFO - Epoch [26][10/33]	lr: 5.000e-04, eta: -1 day, 23:58:40, time: 10.105, data_time: 9.229, memory: 21448, top1_acc: 0.8969, top5_acc: 1.0000, loss_cls: 0.2787, loss: 0.2787, grad_norm: 4.8332
2021-03-22 17:11:02,020 - mmaction - INFO - Epoch [26][20/33]	lr: 5.000e-04, eta: -1 day, 23:57:19, time: 6.847, data_time: 5.975, memory: 21448, top1_acc: 0.9000, top5_acc: 1.0000, loss_cls: 0.2513, loss: 0.2513, grad_norm: 5.1216
2021-03-22 17:12:38,741 - mmaction - INFO - Epoch [26][30/33]	lr: 5.000e-04, eta: -1 day, 23:55:57, time: 9.672, dat

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 126/126, 3.5 task/s, elapsed: 36s, ETA:     0s

2021-03-22 17:32:40,845 - mmaction - INFO - Evaluating top_k_accuracy ...
2021-03-22 17:32:40,847 - mmaction - INFO - 
top1_acc	0.8413
top5_acc	1.0000
2021-03-22 17:32:40,848 - mmaction - INFO - Evaluating mean_class_accuracy ...
2021-03-22 17:32:40,849 - mmaction - INFO - 
mean_acc	0.8413
2021-03-22 17:32:40,850 - mmaction - INFO - Epoch(val) [30][33]	top1_acc: 0.8413, top5_acc: 1.0000, mean_class_accuracy: 0.8413


## Test the trained recognizer

After finetuning the recognizer, let's check the prediction results!

In [14]:
from mmaction.apis import single_gpu_test
from mmaction.datasets import build_dataloader
from mmcv.parallel import MMDataParallel

# Build a test dataloader
dataset = build_dataset(cfg.data.test, dict(test_mode=True))
data_loader = build_dataloader(
        dataset,
        videos_per_gpu=16,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)
model = MMDataParallel(model, device_ids=[0])
outputs = single_gpu_test(model, data_loader)

eval_config = cfg.evaluation
eval_config.pop('interval')
eval_res = dataset.evaluate(outputs, **eval_config)
for name, val in eval_res.items():
    print(f'{name}: {val:.04f}')

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 126/126, 2.1 task/s, elapsed: 59s, ETA:     0s
Evaluating top_k_accuracy ...

top1_acc	0.9048
top5_acc	1.0000

Evaluating mean_class_accuracy ...

mean_acc	0.9048
top1_acc: 0.9048
top5_acc: 1.0000
mean_class_accuracy: 0.9048


# SlowFast

In [25]:
# !mkdir checkpoints
!wget -c https://download.openmmlab.com/   .pth \
      -O checkpoints/db3c461.pth

--2021-03-22 01:46:07--  https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_video_1x1x8_100e_kinetics600_rgb/tsn_r50_video_1x1x8_100e_kinetics600_rgb_20201015-4db3c461.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.75.20.25
Connecting to download.openmmlab.com (download.openmmlab.com)|47.75.20.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99220779 (95M) [application/octet-stream]
Saving to: ‘checkpoints/tsn_r50_video_1x1x8_100e_kinetics600_rgb_20201015-4db3c461.pth’

checkpoints/tsn_r50 100%[===================>]  94,62M  10,5MB/s    in 11s     

2021-03-22 01:46:22 (8,61 MB/s) - ‘checkpoints/tsn_r50_video_1x1x8_100e_kinetics600_rgb_20201015-4db3c461.pth’ saved [99220779/99220779]



In [11]:
from mmcv import Config
cfg = Config.fromfile('./configs/recognition/tsn/tsn_r50_video_1x1x8_100e_kinetics400_rgb.py')

Given a config that trains a TSN model on kinetics400-full dataset, we need to modify some values to use it for training TSN on Kinetics400-tiny dataset.


In [12]:
from mmcv.runner import set_random_seed

# Modify dataset type and path
cfg.dataset_type = 'VideoDataset'
cfg.data_root = 'data/childact_split/train/'
cfg.data_root_val = 'data/childact_split/val/'
cfg.ann_file_train = 'data/childact_split/childact_train_video.txt'
cfg.ann_file_val = 'data/childact_split/childact_val_video.txt'
cfg.ann_file_test = 'data/childact_split/childact_test_video.txt'

cfg.data.test.type = 'VideoDataset'
cfg.data.test.ann_file = 'data/childact_split/childact_test_video.txt'
cfg.data.test.data_prefix = 'data/childact_split/test/'

cfg.data.train.type = 'VideoDataset'
cfg.data.train.ann_file = 'data/childact_split/childact_train_video.txt'
cfg.data.train.data_prefix = 'data/childact_split/train/'

cfg.data.val.type = 'VideoDataset'
cfg.data.val.ann_file = 'data/childact_split/childact_val_video.txt'
cfg.data.val.data_prefix = 'data/childact_split/val/'

# The flag is used to determine whether it is omnisource training
cfg.setdefault('omnisource', False)
# Modify num classes of the model in cls_head
cfg.model.cls_head.num_classes = 7
# We can use the pre-trained TSN model
# cfg.load_from = './checkpoints/best_top1_acc_epoch_5.pth'
cfg.resume_from = './childact-mm/latest.pth'

# Set up working dir to save files and logs.
cfg.work_dir = './childact-mm/'

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.data.videos_per_gpu = 32
cfg.optimizer.lr = 0.0001
# cfg.lr_config.type = 'cyclic'
cfg.total_epochs = 30

# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 2
# We can set the log print interval to reduce the the times of printing log
cfg.log_config.interval = 10

# Set seed thus the results are more reproducible
cfg.seed = 42
set_random_seed(42, deterministic=False)
cfg.gpu_ids = range(1)

# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')


Config:
model = dict(
    type='Recognizer2D',
    backbone=dict(
        type='ResNet',
        pretrained='torchvision://resnet50',
        depth=50,
        norm_eval=False),
    cls_head=dict(
        type='TSNHead',
        num_classes=7,
        in_channels=2048,
        spatial_type='avg',
        consensus=dict(type='AvgConsensus', dim=1),
        dropout_ratio=0.4,
        init_std=0.01),
    train_cfg=None,
    test_cfg=dict(average_clips=None))
optimizer = dict(type='SGD', lr=0.0001, momentum=0.9, weight_decay=0.0001)
optimizer_config = dict(grad_clip=dict(max_norm=40, norm_type=2))
lr_config = dict(policy='step', step=[40, 80])
total_epochs = 30
checkpoint_config = dict(interval=2)
log_config = dict(interval=10, hooks=[dict(type='TextLoggerHook')])
dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = None
resume_from = './childact-mm/latest.pth'
workflow = [('train', 1)]
dataset_type = 'VideoDataset'
data_root = 'data/childact_split/train/'
data_root_val = 'dat

### Train a new recognizer

Finally, lets initialize the dataset and recognizer, then train a new recognizer!

In [13]:
import os.path as osp

from mmaction.datasets import build_dataset
from mmaction.models import build_model
from mmaction.apis import train_model

import mmcv

# Build the dataset
datasets = [build_dataset(cfg.data.train)]

# Build the recognizer
model = build_model(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# model = build_model(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# model = build_detector(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# model = build_detector(cfg.model, train_cfg=cfg.train_cfg, test_cfg=cfg.test_cfg)

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_model(model, datasets, cfg, distributed=False, validate=True)

2021-03-22 16:04:14,045 - mmaction - INFO - These parameters in pretrained checkpoint are not loaded: {'fc.bias', 'fc.weight'}


Use load_from_torchvision loader


2021-03-22 16:04:16,416 - mmaction - INFO - load checkpoint from ./childact-mm/latest.pth
2021-03-22 16:04:16,417 - mmaction - INFO - Use load_from_local loader
2021-03-22 16:04:16,539 - mmaction - INFO - resumed epoch 12, iter 561
2021-03-22 16:04:16,541 - mmaction - INFO - Start running, host: actrec@actrec-HP-Z4-G4-Workstation, work_dir: /home/actrec/.virtualenvs/mmaction/mmaction2/childact-mm
2021-03-22 16:04:16,542 - mmaction - INFO - workflow: [('train', 1)], max: 30 epochs
/home/actrec/.virtualenvs/mmaction/mmaction2/mmaction/core/evaluation/eval_hooks.py:131: UserWarning: runner.meta is None. Creating a empty one.
  warnings.warn('runner.meta is None. Creating a empty one.')
2021-03-22 16:05:56,931 - mmaction - INFO - Epoch [13][10/33]	lr: 5.000e-04, eta: 1:10:06, time: 10.039, data_time: 9.158, memory: 21448, top1_acc: 0.9375, top5_acc: 1.0000, loss_cls: 0.2199, loss: 0.2199, grad_norm: 3.0833
2021-03-22 16:07:05,519 - mmaction - INFO - Epoch [13][20/33]	lr: 5.000e-04, eta: 0:

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 126/126, 3.5 task/s, elapsed: 35s, ETA:     0s

2021-03-22 16:19:08,703 - mmaction - INFO - Evaluating top_k_accuracy ...
2021-03-22 16:19:08,705 - mmaction - INFO - 
top1_acc	0.8492
top5_acc	1.0000
2021-03-22 16:19:08,706 - mmaction - INFO - Evaluating mean_class_accuracy ...
2021-03-22 16:19:08,707 - mmaction - INFO - 
mean_acc	0.8492
2021-03-22 16:19:09,000 - mmaction - INFO - Now best checkpoint is saved as best_top1_acc_epoch_15.pth.
2021-03-22 16:19:09,001 - mmaction - INFO - Best top1_acc is 0.8492 at 15 epoch.
2021-03-22 16:19:09,002 - mmaction - INFO - Epoch(val) [15][33]	top1_acc: 0.8492, top5_acc: 1.0000, mean_class_accuracy: 0.8492
2021-03-22 16:20:54,290 - mmaction - INFO - Epoch [16][10/33]	lr: 5.000e-04, eta: 0:44:12, time: 10.529, data_time: 9.656, memory: 21448, top1_acc: 0.8656, top5_acc: 1.0000, loss_cls: 0.3135, loss: 0.3135, grad_norm: 5.0392
2021-03-22 16:22:01,583 - mmaction - INFO - Epoch [16][20/33]	lr: 5.000e-04, eta: 0:42:08, time: 6.729, data_time: 5.830, memory: 21448, top1_acc: 0.9094, top5_acc: 0.9969,

...

2021-03-22 16:43:42,967 - mmaction - INFO - Evaluating top_k_accuracy ...
2021-03-22 16:43:42,969 - mmaction - INFO - 
top1_acc	0.8571
top5_acc	1.0000
2021-03-22 16:43:42,970 - mmaction - INFO - Evaluating mean_class_accuracy ...
2021-03-22 16:43:42,971 - mmaction - INFO - 
mean_acc	0.8571
2021-03-22 16:43:43,294 - mmaction - INFO - Now best checkpoint is saved as best_top1_acc_epoch_20.pth.
2021-03-22 16:43:43,295 - mmaction - INFO - Best top1_acc is 0.8571 at 20 epoch.
2021-03-22 16:43:43,296 - mmaction - INFO - Epoch(val) [20][33]	top1_acc: 0.8571, top5_acc: 1.0000, mean_class_accuracy: 0.8571
2021-03-22 16:45:20,813 - mmaction - INFO - Epoch [21][10/33]	lr: 5.000e-04, eta: 0:21:02, time: 9.751, data_time: 8.872, memory: 21448, top1_acc: 0.8875, top5_acc: 1.0000, loss_cls: 0.2959, loss: 0.2959, grad_norm: 5.2473
2021-03-22 16:46:30,437 - mmaction - INFO - Epoch [21][20/33]	lr: 5.000e-04, eta: 0:19:34, time: 6.962, data_time: 6.085, memory: 21448, top1_acc: 0.8875, top5_acc: 1.0000, 

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 126/126, 3.5 task/s, elapsed: 36s, ETA:     0s

2021-03-22 17:08:12,494 - mmaction - INFO - Evaluating top_k_accuracy ...
2021-03-22 17:08:12,496 - mmaction - INFO - 
top1_acc	0.8571
top5_acc	1.0000
2021-03-22 17:08:12,496 - mmaction - INFO - Evaluating mean_class_accuracy ...
2021-03-22 17:08:12,497 - mmaction - INFO - 
mean_acc	0.8571
2021-03-22 17:08:12,497 - mmaction - INFO - Epoch(val) [25][33]	top1_acc: 0.8571, top5_acc: 1.0000, mean_class_accuracy: 0.8571
2021-03-22 17:09:53,555 - mmaction - INFO - Epoch [26][10/33]	lr: 5.000e-04, eta: -1 day, 23:58:40, time: 10.105, data_time: 9.229, memory: 21448, top1_acc: 0.8969, top5_acc: 1.0000, loss_cls: 0.2787, loss: 0.2787, grad_norm: 4.8332
2021-03-22 17:11:02,020 - mmaction - INFO - Epoch [26][20/33]	lr: 5.000e-04, eta: -1 day, 23:57:19, time: 6.847, data_time: 5.975, memory: 21448, top1_acc: 0.9000, top5_acc: 1.0000, loss_cls: 0.2513, loss: 0.2513, grad_norm: 5.1216
2021-03-22 17:12:38,741 - mmaction - INFO - Epoch [26][30/33]	lr: 5.000e-04, eta: -1 day, 23:55:57, time: 9.672, dat

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 126/126, 3.5 task/s, elapsed: 36s, ETA:     0s

2021-03-22 17:32:40,845 - mmaction - INFO - Evaluating top_k_accuracy ...
2021-03-22 17:32:40,847 - mmaction - INFO - 
top1_acc	0.8413
top5_acc	1.0000
2021-03-22 17:32:40,848 - mmaction - INFO - Evaluating mean_class_accuracy ...
2021-03-22 17:32:40,849 - mmaction - INFO - 
mean_acc	0.8413
2021-03-22 17:32:40,850 - mmaction - INFO - Epoch(val) [30][33]	top1_acc: 0.8413, top5_acc: 1.0000, mean_class_accuracy: 0.8413


## Test the trained recognizer

After finetuning the recognizer, let's check the prediction results!

In [14]:
from mmaction.apis import single_gpu_test
from mmaction.datasets import build_dataloader
from mmcv.parallel import MMDataParallel

# Build a test dataloader
dataset = build_dataset(cfg.data.test, dict(test_mode=True))
data_loader = build_dataloader(
        dataset,
        videos_per_gpu=16,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)
model = MMDataParallel(model, device_ids=[0])
outputs = single_gpu_test(model, data_loader)

eval_config = cfg.evaluation
eval_config.pop('interval')
eval_res = dataset.evaluate(outputs, **eval_config)
for name, val in eval_res.items():
    print(f'{name}: {val:.04f}')

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 126/126, 2.1 task/s, elapsed: 59s, ETA:     0s
Evaluating top_k_accuracy ...

top1_acc	0.9048
top5_acc	1.0000

Evaluating mean_class_accuracy ...

mean_acc	0.9048
top1_acc: 0.9048
top5_acc: 1.0000
mean_class_accuracy: 0.9048
